Importe o comet_ml ante de importar o pytorch

In [1]:
from comet_ml import Experiment

In [2]:
import torch

Caso esteja usando cuda

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

 Quando terminar de rodar um link vai ficar disponivel onde os dados estão sendo salvos.

Controle de algumas variaveis. Isso é muito util para comparar modelos! 
Tente rodar uma vez e depois mude o learning_rate e rode denovo, dois experimentos serão criado

In [4]:
experiment = Experiment(
    api_key="your_key",
    project_name="",
    workspace="",
)
hyper_params = {
    "learning_rate": 0.0001,
    "epochs": 5,
    "batch_size": 100,
}
experiment.log_parameters(hyper_params)

COMET ERROR: The given API key your_key is invalid, please check it against the dashboard. Your experiment would not be logged 
For more details, please refer to: https://www.comet.ml/docs/python-sdk/warnings-errors/


Baixando os dados do MNIST e criando loaders de train e test, repare que usamos o hyper_params['batch_size'] aqui. 

In [5]:
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
train_data = datasets.MNIST(
    root = 'data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = 'data', 
    train = False, 
    transform = ToTensor()
)

loaders = {
    'train' : torch.utils.data.DataLoader(train_data, 
                                          batch_size=hyper_params['batch_size'], 
                                          shuffle=True, 
                                          num_workers=1),
    
    'test'  : torch.utils.data.DataLoader(test_data, 
                                          batch_size=hyper_params['batch_size'], 
                                          shuffle=True, 
                                          num_workers=1),
}
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x212041444f0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x212041446d0>}

In [6]:
import torch.nn as nn
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         
            nn.Conv2d(
                in_channels=1,              
                out_channels=16,            
                kernel_size=5,              
                stride=1,                   
                padding=2,                  
            ),                              
            nn.ReLU(),                      
            nn.MaxPool2d(kernel_size=2),    
        )
        self.conv2 = nn.Sequential(         
            nn.Conv2d(16, 32, 5, 1, 2),     
            nn.ReLU(),                      
            nn.MaxPool2d(2),                
        )
        # fully connected layer, output 10 classes
        self.out = nn.Linear(32 * 7 * 7, 10)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        # flatten the output of conv2 to (batch_size, 32 * 7 * 7)
        x = x.view(x.size(0), -1)       
        output = self.out(x)
        return output, x    # return x for visualization
cnn = CNN()

função loss e otimizador 

In [7]:
from torch import optim
loss_func = nn.CrossEntropyLoss()   
optimizer = optim.Adam(cnn.parameters(), lr = hyper_params['learning_rate'])   

Nesse treino log_metric vai criar um grafico da metrica pelo o que passarmos no terceiro parametro, no caso: run

In [8]:
from torch.autograd import Variable
def train(num_epochs, cnn, loaders):
    
    cnn.train()
        
    # Train the model
    total_step = len(loaders['train'])
    run = 0
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(loaders['train']):
            run = run + 1
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(images)   # batch x
            b_y = Variable(labels)   # batch y
            output = cnn(b_x)[0]               
            loss = loss_func(output, b_y)
            
            # clear gradients for this training step   
            optimizer.zero_grad()           
            
            # backpropagation, compute gradients 
            loss.backward()    
            # apply gradients             
            optimizer.step()                
            experiment.log_metric("loss", loss.item(), run)
            if (i+1) % 100 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
        
        pass
    
    
    pass
train(hyper_params['epochs'], cnn, loaders)

Epoch [1/5], Step [100/600], Loss: 1.8551
Epoch [1/5], Step [200/600], Loss: 0.8252
Epoch [1/5], Step [300/600], Loss: 0.5959
Epoch [1/5], Step [400/600], Loss: 0.2785
Epoch [1/5], Step [500/600], Loss: 0.4235
Epoch [1/5], Step [600/600], Loss: 0.4099
Epoch [2/5], Step [100/600], Loss: 0.2951
Epoch [2/5], Step [200/600], Loss: 0.2068
Epoch [2/5], Step [300/600], Loss: 0.1953
Epoch [2/5], Step [400/600], Loss: 0.2425
Epoch [2/5], Step [500/600], Loss: 0.4083
Epoch [2/5], Step [600/600], Loss: 0.3104
Epoch [3/5], Step [100/600], Loss: 0.2818
Epoch [3/5], Step [200/600], Loss: 0.1602
Epoch [3/5], Step [300/600], Loss: 0.1371
Epoch [3/5], Step [400/600], Loss: 0.1850
Epoch [3/5], Step [500/600], Loss: 0.2228
Epoch [3/5], Step [600/600], Loss: 0.1161
Epoch [4/5], Step [100/600], Loss: 0.1661
Epoch [4/5], Step [200/600], Loss: 0.1672
Epoch [4/5], Step [300/600], Loss: 0.1250
Epoch [4/5], Step [400/600], Loss: 0.2298
Epoch [4/5], Step [500/600], Loss: 0.1448
Epoch [4/5], Step [600/600], Loss:

Fazemos, por ultimo, outra metrica, mas dessa vez para a acurácia 

In [9]:
def test():
    # Test the model
    cnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for num,(images, labels) in enumerate(loaders['test']):
            test_output, last_layer = cnn(images)
            pred_y = torch.max(test_output, 1)[1].data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
            experiment.log_metric("accuracy", accuracy, num)
        print('Test Accuracy of the model on the 10000 test images: %.2f' % accuracy)
    
test()

Test Accuracy of the model on the 10000 test images: 0.95
